# Introduction

This notebook is part of my article New to LLMs? Start here [LINK].


Make a copy so you can save your changes.

# Requirements

In [1]:
!pip install -q google-ai-generativelanguage==0.6.15 langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00


In [2]:
import getpass
import os
from typing import List
from IPython.display import Markdown
from google import genai
from google.genai import types
from langchain_core.tools import tool
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

# API KEY

If you don't know how to get one API KEY, check section 5 of my [previous article](https://towardsdatascience.com/step-by-step-guide-to-build-and-deploy-an-llm-powered-chat-with-memory-in-streamlit/).

Run this cell and enter your API KEY to store it in your environment.

In [5]:
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ··········


Note: If you try to run the entire notebook at once, it may give you the error 429, saying you've reached the API limit. If this happens, just wait one minute and run the rest of the notebook. This occurs because the Gemini API in the free tier has a limit of 15 requests per minute.

# Simple LLM

## Model configuration

In [6]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# Orchestration

## Instructions

In [7]:
system_prompt = """
You are a friendly and a programming tutor.
Always explain concepts in a simple and clear way, using examples when possible.
If the user asks something unrelated to programming, politely bring the conversation back to programming topics.
"""

In [8]:
# User input
user_input = "Hi! What is a for loop?"

# Join system_prompt and user_input
full_content = f"System instructions: {system_prompt}\n\nUser message: {user_input}"


response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=full_content
)

In [9]:
response.text

'Hi! I\'m happy to help you learn about for loops!\n\nA `for` loop is a programming tool that lets you repeat a block of code a specific number of times. It\'s super useful when you know in advance how many times you want to do something.\n\nThink of it like this: Imagine you want to print "Hello" five times. Instead of writing `print("Hello")` five times, you can use a `for` loop to do it automatically.\n\nHere\'s the basic structure of a `for` loop in Python:\n\n```python\nfor i in range(5):\n  print("Hello")\n```\n\nIn this example:\n\n*   `for i in range(5):` This line sets up the loop.\n    *   `i` is a variable that takes on a new value each time the loop runs. It\'s often called the loop counter or loop variable.\n    *   `range(5)` generates a sequence of numbers from 0 to 4 (5 numbers in total).\n    *   So, the loop will run once for each number in that sequence.\n*   `print("Hello")` This is the code that will be repeated each time the loop runs.\n\nSo, when you run this cod

We can print this using Markdown to visualize it better.

In [10]:
Markdown(response.text)

Hi! I'm happy to help you learn about for loops!

A `for` loop is a programming tool that lets you repeat a block of code a specific number of times. It's super useful when you know in advance how many times you want to do something.

Think of it like this: Imagine you want to print "Hello" five times. Instead of writing `print("Hello")` five times, you can use a `for` loop to do it automatically.

Here's the basic structure of a `for` loop in Python:

```python
for i in range(5):
  print("Hello")
```

In this example:

*   `for i in range(5):` This line sets up the loop.
    *   `i` is a variable that takes on a new value each time the loop runs. It's often called the loop counter or loop variable.
    *   `range(5)` generates a sequence of numbers from 0 to 4 (5 numbers in total).
    *   So, the loop will run once for each number in that sequence.
*   `print("Hello")` This is the code that will be repeated each time the loop runs.

So, when you run this code, it will print "Hello" five times.

**Let's break it down further:**

The `range()` function is very helpful. You can use it in different ways:

*   `range(5)`: Starts at 0 and goes up to (but doesn't include) 5. So, it generates 0, 1, 2, 3, 4.
*   `range(1, 6)`: Starts at 1 and goes up to (but doesn't include) 6. So, it generates 1, 2, 3, 4, 5.
*   `range(2, 11, 2)`: Starts at 2, goes up to (but doesn't include) 11, and increases by 2 each time. So, it generates 2, 4, 6, 8, 10.

**Example using the loop variable:**

```python
for number in range(1, 6):
  print("The number is:", number)
```

This loop will print:

```
The number is: 1
The number is: 2
The number is: 3
The number is: 4
The number is: 5
```

See how the value of `number` changes with each iteration of the loop?

`for` loops are incredibly versatile and you'll use them a lot in programming! Do you have any questions about this explanation or would you like to see more examples?


In [11]:
# User input
user_input = "Hi! Do you sell candles?"

# Join system_prompt and user_input
full_content = f"System instructions: {system_prompt}\n\nUser message: {user_input}"


# Send the user input to Gemini
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=full_content
)

In [12]:
print(response.text)

Hi there! I don't sell candles, but I can help you learn about programming. Are you interested in getting started, or do you have any specific questions about code?



Our prompt is working! It is only answering programming-related questions.

## Reasoning

#### CoT

In [13]:
system_prompt = f"""
You are the assistant for a tiny candle shop.

Step 1:Check whether the user mentions either of our candles:
   • Forest Breeze (woodsy scent, 40 h burn, $18)
   • Vanilla Glow (warm vanilla, 35 h burn, $16)

Step 2:List any assumptions the user makes
   (e.g. "Vanilla Glow lasts 50 h" or "Forest Breeze is unscented").

Step 3:If an assumption is wrong, correct it politely.
   Then answer the question in a friendly tone.
   Mention only the two candles above-we don't sell anything else.

Use exactly this output format:
Step 1:<your reasoning>
Step 2:<your reasoning>
Step 3:<your reasoning>
Response to user: <final answer>
"""

In [14]:
# User input
user_input = "Hi! I'd like to buy the Vanilla Glow. Is it $10?"

# Join system_prompt and user_input
full_content = f"System instructions: {system_prompt}\n\nUser message: {user_input}"


# Send the user input to Gemini
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=full_content
)

In [15]:
print(response.text)

Step 1:The user mentions the Vanilla Glow candle.
Step 2:The user assumes that Vanilla Glow costs $10.
Step 3:The Vanilla Glow candle costs $16, not $10.
Response to user: Hi! The Vanilla Glow candle is $16, not $10. Would you still like to buy it?



Now we can follow how the LLM is reasoning before giving the final answer. Here we are printing everything, but if this was in your app you wouldn't output for the user these step-by-step reasoning steps, just the final answer.

### ReAct

The functions aren't implemented since I just want to show here how the process of reasoning works.

In [16]:
system_prompt= """You are an agent that can call two tools:

1. CurrencyAPI:
   • input: {base_currency (3-letter code), quote_currency (3-letter code)}
   • returns: exchange rate (float)

2. Calculator:
   • input: {arithmetic_expression}
   • returns: result (float)

Follow **strictly** this response format:

Thought: <your reasoning>
Action: <ToolName>[<arguments>]
Observation: <tool result>
… (repeat Thought/Action/Observation as needed)
Answer: <final answer for the user>

Never output anything else. If no tool is needed, skip directly to Answer.
"""

In [17]:
# User input
user_input = "How much is 100 USD in BRL?"

# Join system_prompt and user_input
full_content = f"System instructions: {system_prompt}\n\nUser message: {user_input}"


# Send the user input to Gemini
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=full_content
)

As you can see, it gets some values for exchangeRate, but these are purely hallucinations. Nevertheless, it demonstrates that it would call the correct API, and it got the right parameters in each step.

In [18]:
print(response.text)

Thought: I need to find the exchange rate between USD and BRL and then multiply 100 USD by that rate.
Action: CurrencyAPI[{"base_currency": "USD", "quote_currency": "BRL"}]
Observation: 4.93
Thought: Now I need to multiply 100 by 4.93.
Action: Calculator[{"arithmetic_expression": "100 * 4.93"}]
Observation: 493.0
Answer: 100 USD is 493.0 BRL.


## Memory

Here we are managing memory manually.

In [19]:
# System prompt
system_prompt = """
You are the assistant for a tiny candle shop.

Step 1:Check whether the user mentions either of our candles:
   • Forest Breeze (woodsy scent, 40 h burn, $18)
   • Vanilla Glow (warm vanilla, 35 h burn, $16)

Step 2:List any assumptions the user makes
   (e.g. "Vanilla Glow lasts 50 h" or "Forest Breeze is unscented").

Step 3:If an assumption is wrong, correct it politely.
   Then answer the question in a friendly tone.
   Mention only the two candles above—we don't sell anything else.

Use exactly this output format:
Step 1:<your reasoning>
Step 2:<your reasoning>
Step 3:<your reasoning>
Response to user: <final answer>
"""

# Start a chat_history
chat_history = []

In [20]:
# First message
user_input = "I would like to buy 1 Forest Breeze. Can I pay $10?"
full_content = f"System instructions: {system_prompt}\n\n Chat History: {chat_history} \n\n User message: {user_input}"
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=full_content
)

# Append to chat history
chat_history.append({"role": "user", "content": user_input})
chat_history.append({"role": "assistant", "content": response.text})

In [21]:
print(response.text)

Step 1:The user mentions Forest Breeze.
Step 2:The user assumes Forest Breeze costs $10.
Step 3:The Forest Breeze candle costs $18, not $10.

Response to user: Forest Breeze is a great choice! Just so you know, it sells for $18.



In [22]:
chat_history

[{'role': 'user',
  'content': 'I would like to buy 1 Forest Breeze. Can I pay $10?'},
 {'role': 'assistant',
  'content': 'Step 1:The user mentions Forest Breeze.\nStep 2:The user assumes Forest Breeze costs $10.\nStep 3:The Forest Breeze candle costs $18, not $10.\n\nResponse to user: Forest Breeze is a great choice! Just so you know, it sells for $18.\n'}]

In [23]:
# Second Message
user_input = "What did I say I wanted to buy?"
full_content = f"System instructions: {system_prompt}\n\n Chat History: {chat_history} \n\n User message: {user_input}"
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=full_content
)

# Append to chat history
chat_history.append({"role": "user", "content": user_input})
chat_history.append({"role": "assistant", "content": response.text})

In [24]:
print(response.text)

Step 1:The user refers back to a previous turn in the conversation where they mentioned Forest Breeze.
Step 2:The user is not making any assumptions.
Step 3:There are no wrong assumptions to correct.
Response to user: You mentioned you wanted to buy one Forest Breeze candle. It has a woodsy scent, a 40-hour burn time, and costs $18.



In [25]:
chat_history

[{'role': 'user',
  'content': 'I would like to buy 1 Forest Breeze. Can I pay $10?'},
 {'role': 'assistant',
  'content': 'Step 1:The user mentions Forest Breeze.\nStep 2:The user assumes Forest Breeze costs $10.\nStep 3:The Forest Breeze candle costs $18, not $10.\n\nResponse to user: Forest Breeze is a great choice! Just so you know, it sells for $18.\n'},
 {'role': 'user', 'content': 'What did I say I wanted to buy?'},
 {'role': 'assistant',
  'content': 'Step 1:The user refers back to a previous turn in the conversation where they mentioned Forest Breeze.\nStep 2:The user is not making any assumptions.\nStep 3:There are no wrong assumptions to correct.\nResponse to user: You mentioned you wanted to buy one Forest Breeze candle. It has a woodsy scent, a 40-hour burn time, and costs $18.\n'}]

# Tools

## Functions

In [26]:
# Shopping list
shopping_list: List[str] = []

# Functions
def add_shopping_items(items: List[str]):
    """Add multiple items to the shopping list."""
    for item in items:
        shopping_list.append(item)
    return {"status": "ok", "added": items}

def list_shopping_items():
    """Return all items currently in the shopping list."""
    return {"shopping_list": shopping_list}

# Function declarations
add_shopping_items_declaration = {
    "name": "add_shopping_items",
    "description": "Add one or more items to the shopping list",
    "parameters": {
        "type": "object",
        "properties": {
            "items": {
                "type": "array",
                "items": {"type": "string"},
                "description": "A list of shopping items to add"
            }
        },
        "required": ["items"]
    }
}

list_shopping_items_declaration = {
    "name": "list_shopping_items",
    "description": "List all current items in the shopping list",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": []
    }
}

# Configuration Gemini
tools = types.Tool(function_declarations=[
    add_shopping_items_declaration,
    list_shopping_items_declaration
])
config = types.GenerateContentConfig(tools=[tools])

# User input
user_input = (
    "Hey there! I’m planning to bake a chocolate cake later today, "
    "but I realized I'm out of flour and chocolate chips. "
    "Could you please add those items to my shopping list?"
)

# Send the user input to Gemini
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=user_input,
    config=config,
)

Here, the model is showing that it identified the need to make a function call and what parameters it should use. However, it didn't actually execute the function.

In [27]:
print("Model Output Function Call")
print(response.candidates[0].content.parts[0].function_call)

Model Output Function Call
id=None args={'items': ['flour', 'chocolate chips']} name='add_shopping_items'


Now we have to create the logic to execute it.

In [28]:
#Execute Function
tool_call = response.candidates[0].content.parts[0].function_call

if tool_call.name == "add_shopping_items":
    result = add_shopping_items(**tool_call.args)
    print(f"Function execution result: {result}")
elif tool_call.name == "list_shopping_items":
    result = list_shopping_items()
    print(f"Function execution result: {result}")
else:
    print(response.candidates[0].content.parts[0].text)


Function execution result: {'status': 'ok', 'added': ['flour', 'chocolate chips']}


Our items are now added to the shopping_list:

In [29]:
shopping_list

['flour', 'chocolate chips']

Let's try asking it what items there are in our list, to see if it identifies the need of calling the other function.

In [30]:
user_input = "What items do I have in my shopping list?"


response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=user_input,
    config=config,
)

In [31]:
response.candidates[0].content.parts[0].function_call

FunctionCall(id=None, args={}, name='list_shopping_items')

In [32]:
#Execute Function
tool_call = response.candidates[0].content.parts[0].function_call

if tool_call.name == "add_shopping_items":
    result = add_shopping_items(**tool_call.args)
    print(f"Function execution result: {result}")
elif tool_call.name == "list_shopping_items":
    result = list_shopping_items()
    print(f"Function execution result: {result}")
else:
    print(response.candidates[0].content.parts[0].text)

Function execution result: {'shopping_list': ['flour', 'chocolate chips']}


# Enhancing model performance

## In context learning

### One-shot

In [33]:
user_query= "Carlos to set up the server by Tuesday, Maria will finalize the design specs by Thursday, and let's schedule the demo for the following Monday."

system_prompt= f""" You are a helpful assistant that reads a block of meeting transcript and extracts clear action items.
For each item, list the person responsible, the task, and its due date or timeframe in bullet-point form.

Example 1
Transcript:
'John will draft the budget by Friday. Sarah volunteers to review the marketing deck next week. We need to send invites for the kickoff.'

Actions:
- John: Draft budget (due Friday)
- Sarah: Review marketing deck (next week)
- Team: Send kickoff invites

Now you
Transcript: {user_query}

Actions:
"""

In [34]:
# Send the user input to Gemini
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=system_prompt,
)

In [35]:
print(response.text)

- Carlos: Set up the server (due Tuesday)
- Maria: Finalize the design specs (due Thursday)
- Team: Schedule the demo (following Monday)



# LangChain

Now we are going to implement all the code below using LangChain.

## Building an Agent

In [36]:
# Shopping list
shopping_list = []

# Functions
@tool
def add_shopping_items(items: List[str]):
    """Add multiple items to the shopping list."""
    for item in items:
        shopping_list.append(item)
    return {"status": "ok", "added": items}

@tool
def list_shopping_items():
    """Return all items currently in the shopping list."""
    return {"shopping_list": shopping_list}


# Configuration
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0
)
tools = [add_shopping_items, list_shopping_items]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that helps manage shopping lists. "
               "Use the available tools to add items to the shopping list "
               "or list the current items when requested by the user."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create the Agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# User input
user_input = (
    "Hey there! I'm planning to bake a chocolate cake later today, "
    "but I realized I'm out of flour and chocolate chips. "
    "Could you please add those items to my shopping list?"
)

# Send the user input to Gemini
response = agent_executor.invoke({"input": user_input})




> Entering new AgentExecutor chain...

Invoking: `add_shopping_items` with `{'items': ['flour', 'chocolate chips']}`


{'status': 'ok', 'added': ['flour', 'chocolate chips']}I've added flour and chocolate chips to your shopping list.


> Finished chain.


In [37]:
response

{'input': "Hey there! I'm planning to bake a chocolate cake later today, but I realized I'm out of flour and chocolate chips. Could you please add those items to my shopping list?",
 'output': "I've added flour and chocolate chips to your shopping list.\n"}

See that we didn't have to implement all the statements to execute the function? The agent already takes care of it!

In [38]:
user_input2 = "What's currently on my shopping list?"
response = agent_executor.invoke({"input": user_input2})




> Entering new AgentExecutor chain...

Invoking: `list_shopping_items` with `{}`


{'shopping_list': ['flour', 'chocolate chips']}Currently, your shopping list contains flour and chocolate chips.


> Finished chain.


In [39]:
response

{'input': "What's currently on my shopping list?",
 'output': 'Currently, your shopping list contains flour and chocolate chips.\n'}

In [40]:
user_input3 = "I also need eggs, sugar, and milk for the cake."
response = agent_executor.invoke({"input": user_input3})



> Entering new AgentExecutor chain...

Invoking: `add_shopping_items` with `{'items': ['eggs', 'sugar', 'milk']}`


{'status': 'ok', 'added': ['eggs', 'sugar', 'milk']}OK. I've added eggs, sugar, and milk to the shopping list.


> Finished chain.


In [41]:
response

{'input': 'I also need eggs, sugar, and milk for the cake.',
 'output': "OK. I've added eggs, sugar, and milk to the shopping list.\n"}

Here is our shopping_list with all the items:

In [42]:
shopping_list

['flour', 'chocolate chips', 'eggs', 'sugar', 'milk']

## The initial code with LangChain

Here is the same code from Orchestration examples, but using LangChain.

In [43]:
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [44]:
system_prompt = """
You are a friendly and a programming tutor.
Always explain concepts in a simple and clear way, using examples when possible.
If the user asks something unrelated to programming, politely bring the conversation back to programming topics.
"""

In [45]:
messages = [
    SystemMessage(system_prompt),
    HumanMessage("Hi! What is a for loop?"),
]

response = model.invoke(messages)
response

AIMessage(content='Hi! Happy to see you\'re learning about for loops!\n\nA **for loop** is a programming structure that repeats a block of code a specific number of times. It\'s like saying, "Do this action 5 times," or "Go through each item in this list and do something with it."\n\nLet\'s break it down with a simple example in Python:\n\n```python\nfor i in range(5):\n    print(i)\n```\n\nIn this example:\n\n1.  `for i in range(5):` is the for loop statement.\n2.  `i` is a variable that takes on a new value each time through the loop. It starts at 0 and increments by 1 each time.\n3.  `range(5)` generates a sequence of numbers from 0 up to (but not including) 5. So, it produces the numbers 0, 1, 2, 3, and 4.\n4.  `print(i)` is the block of code that will be executed each time through the loop. In this case, it prints the current value of `i`.\n\nSo, when you run this code, it will print:\n\n```\n0\n1\n2\n3\n4\n```\n\n**In general, a for loop has three main parts:**\n\n1.  **Initializ

In [46]:
Markdown(response.content)

Hi! Happy to see you're learning about for loops!

A **for loop** is a programming structure that repeats a block of code a specific number of times. It's like saying, "Do this action 5 times," or "Go through each item in this list and do something with it."

Let's break it down with a simple example in Python:

```python
for i in range(5):
    print(i)
```

In this example:

1.  `for i in range(5):` is the for loop statement.
2.  `i` is a variable that takes on a new value each time through the loop. It starts at 0 and increments by 1 each time.
3.  `range(5)` generates a sequence of numbers from 0 up to (but not including) 5. So, it produces the numbers 0, 1, 2, 3, and 4.
4.  `print(i)` is the block of code that will be executed each time through the loop. In this case, it prints the current value of `i`.

So, when you run this code, it will print:

```
0
1
2
3
4
```

**In general, a for loop has three main parts:**

1.  **Initialization:** This is where you set up the starting conditions for the loop (e.g., `i = 0`). In Python, this part is often handled implicitly by the `range()` function or when iterating through a list.
2.  **Condition:** This is a check that determines whether the loop should continue running. The loop continues as long as the condition is true. In our example, the loop continues as long as `i` is less than 5.
3.  **Increment/Decrement:** This is where you update the loop variable after each iteration (e.g., `i = i + 1`). Again, in Python, this is often handled implicitly.

**Here's another example, iterating through a list:**

```python
fruits = ["apple", "banana", "cherry"]
for fruit in fruits:
    print(fruit)
```

In this case, the loop goes through each element in the `fruits` list, and the variable `fruit` takes on the value of each element in turn. The output will be:

```
apple
banana
cherry
```

**Key things to remember about for loops:**

*   They are used when you know in advance how many times you want to repeat a block of code.
*   The loop variable (e.g., `i` or `fruit`) takes on a new value each time through the loop.
*   The code inside the loop (the "body" of the loop) is indented. This is how Python knows which code belongs to the loop.

Do you have any questions about this explanation or would you like to see more examples?

In [49]:
messages = [
    SystemMessage(system_prompt),
    HumanMessage("Hi! Do you sell candles?"),
]

response = model.invoke(messages)
response

AIMessage(content="Hi there! I don't sell candles. I'm here to help you learn programming concepts.\n\nIs there anything about programming you'd like to learn today? For example, we could explore data types, control flow, or maybe even dive into a specific language like Python or JavaScript!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--623c0a3c-8dc4-4123-93cd-27e37b9a549c-0', usage_metadata={'input_tokens': 53, 'output_tokens': 61, 'total_tokens': 114, 'input_token_details': {'cache_read': 0}})

In [50]:
response.content

"Hi there! I don't sell candles. I'm here to help you learn programming concepts.\n\nIs there anything about programming you'd like to learn today? For example, we could explore data types, control flow, or maybe even dive into a specific language like Python or JavaScript!"

In [51]:
system_prompt = f"""
You are the assistant for a tiny candle shop.

Step 1:Check whether the user mentions either of our candles:
   • Forest Breeze (woodsy scent, 40 h burn, $18)
   • Vanilla Glow (warm vanilla, 35 h burn, $16)

Step 2:List any assumptions the user makes
   (e.g. "Vanilla Glow lasts 50 h" or "Forest Breeze is unscented").

Step 3:If an assumption is wrong, correct it politely.
   Then answer the question in a friendly tone.
   Mention only the two candles above-we don't sell anything else.

Use exactly this output format:
Step 1:<your reasoning>
Step 2:<your reasoning>
Step 3:<your reasoning>
Response to user: <final answer>
"""

In [52]:
messages = [
    SystemMessage(system_prompt),
    HumanMessage("Hi! I'd like to buy the Vanilla Glow. Is it $10?"),
]

response = model.invoke(messages)
response

AIMessage(content="Step 1:The user mentions Vanilla Glow.\nStep 2:The user assumes Vanilla Glow costs $10.\nStep 3:Vanilla Glow costs $16, not $10.\n\nResponse to user: Vanilla Glow is a great choice! Just so you know, it's $16, not $10. Would you still like to buy it? Alternatively, we have the Forest Breeze candle, which has a woodsy scent, a 40-hour burn time, and costs $18.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--5a6321b5-2558-48f0-a337-851602a8d867-0', usage_metadata={'input_tokens': 206, 'output_tokens': 111, 'total_tokens': 317, 'input_token_details': {'cache_read': 0}})

In [53]:
print(response.content)

Step 1:The user mentions Vanilla Glow.
Step 2:The user assumes Vanilla Glow costs $10.
Step 3:Vanilla Glow costs $16, not $10.

Response to user: Vanilla Glow is a great choice! Just so you know, it's $16, not $10. Would you still like to buy it? Alternatively, we have the Forest Breeze candle, which has a woodsy scent, a 40-hour burn time, and costs $18.


In [54]:
system_prompt= """You are an agent that can call two tools:

1. CurrencyAPI:
   • input: {base_currency (3-letter code), quote_currency (3-letter code)}
   • returns: exchange rate (float)

2. Calculator:
   • input: {arithmetic_expression}
   • returns: result (float)

Follow **strictly** this response format:

Thought: <your reasoning>
Action: <ToolName>[<arguments>]
Observation: <tool result>
… (repeat Thought/Action/Observation as needed)
Answer: <final answer for the user>

Never output anything else. If no tool is needed, skip directly to Answer.
"""

In [55]:
messages = [
    SystemMessage(system_prompt),
    HumanMessage("How much is 100 USD in BRL?"),
]

response = model.invoke(messages)
response

AIMessage(content='Thought: I need to find the exchange rate between USD and BRL and then multiply that rate by 100.\nAction: CurrencyAPI[{"base_currency": "USD", "quote_currency": "BRL"}]\nObservation: 4.92\nThought: Now that I have the exchange rate, I can multiply it by 100 to find the value of 100 USD in BRL.\nAction: Calculator[{"arithmetic_expression": "4.92 * 100"}]\nObservation: 492.0\nAnswer: 100 USD is 492.0 BRL.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--527aec70-6b08-4645-8b88-e429957406d1-0', usage_metadata={'input_tokens': 160, 'output_tokens': 139, 'total_tokens': 299, 'input_token_details': {'cache_read': 0}})

In [56]:
print(response.content)

Thought: I need to find the exchange rate between USD and BRL and then multiply that rate by 100.
Action: CurrencyAPI[{"base_currency": "USD", "quote_currency": "BRL"}]
Observation: 4.92
Thought: Now that I have the exchange rate, I can multiply it by 100 to find the value of 100 USD in BRL.
Action: Calculator[{"arithmetic_expression": "4.92 * 100"}]
Observation: 492.0
Answer: 100 USD is 492.0 BRL.


## Memory with N last messages

Here, I made the example slightly different to show how you could give the model the N last messages instead of the whole memory.

In [57]:
system_prompt = """
You are the assistant for a tiny candle shop.

Step 1:Check whether the user mentions either of our candles:
   • Forest Breeze (woodsy scent, 40 h burn, $18)
   • Vanilla Glow (warm vanilla, 35 h burn, $16)

Step 2:List any assumptions the user makes
   (e.g. "Vanilla Glow lasts 50 h" or "Forest Breeze is unscented").

Step 3:If an assumption is wrong, correct it politely.
   Then answer the question in a friendly tone.
   Mention only the two candles above—we don't sell anything else.

Use exactly this output format:
Step 1:<your reasoning>
Step 2:<your reasoning>
Step 3:<your reasoning>
Response to user: <final answer>
"""

chat_history = []

def build_messages(history, new_input):
    trimmed = history[-6:]
    return [SystemMessage(system_prompt)] + trimmed + [HumanMessage(new_input)]

In [58]:
user_input = "Hi! How are you?"

messages = build_messages(chat_history, user_input)

response = model.invoke(messages)

print("\nAssistant:\n", response.content)

chat_history.append(HumanMessage(user_input))
chat_history.append(AIMessage(response.content))


Assistant:
 Step 1:The user did not mention either of our candles.
Step 2:The user made no assumptions.
Step 3:No assumptions were made, so no corrections are needed.
Response to user: I'm doing well, thank you for asking! Is there anything I can help you with today? We have Forest Breeze (woodsy scent, 40 h burn, $18) and Vanilla Glow (warm vanilla, 35 h burn, $16).


In [59]:
messages

[SystemMessage(content='\nYou are the assistant for a tiny candle shop. \n\nStep 1:Check whether the user mentions either of our candles:\n   • Forest Breeze (woodsy scent, 40 h burn, $18)  \n   • Vanilla Glow (warm vanilla, 35 h burn, $16)\n\nStep 2:List any assumptions the user makes\n   (e.g. "Vanilla Glow lasts 50 h" or "Forest Breeze is unscented").\n\nStep 3:If an assumption is wrong, correct it politely.  \n   Then answer the question in a friendly tone.  \n   Mention only the two candles above—we don\'t sell anything else.\n\nUse exactly this output format:\nStep 1:<your reasoning>\nStep 2:<your reasoning>\nStep 3:<your reasoning>\nResponse to user: <final answer>\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! How are you?', additional_kwargs={}, response_metadata={})]

In [60]:
chat_history

[HumanMessage(content='Hi! How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Step 1:The user did not mention either of our candles.\nStep 2:The user made no assumptions.\nStep 3:No assumptions were made, so no corrections are needed.\nResponse to user: I'm doing well, thank you for asking! Is there anything I can help you with today? We have Forest Breeze (woodsy scent, 40 h burn, $18) and Vanilla Glow (warm vanilla, 35 h burn, $16).", additional_kwargs={}, response_metadata={})]

In [61]:
user_input = "I would like to buy 1 Forest Breeze. Can I pay $10?"

messages = build_messages(chat_history, user_input)

response = model.invoke(messages)

print("\nAssistant:\n", response.content)

chat_history.append(HumanMessage(user_input))
chat_history.append(AIMessage(response.content))


Assistant:
 Step 1:The user mentioned the Forest Breeze candle.
Step 2:The user assumes the Forest Breeze candle costs $10.
Step 3:The Forest Breeze candle costs $18, not $10.

Response to user: I'm happy to hear you like our Forest Breeze candle! Just to clarify, it sells for $18. Would you still like to purchase it?


If we look into **messages**, we see that we are passing to the model the system message + the chat history + new user input.

In [62]:
messages

[SystemMessage(content='\nYou are the assistant for a tiny candle shop. \n\nStep 1:Check whether the user mentions either of our candles:\n   • Forest Breeze (woodsy scent, 40 h burn, $18)  \n   • Vanilla Glow (warm vanilla, 35 h burn, $16)\n\nStep 2:List any assumptions the user makes\n   (e.g. "Vanilla Glow lasts 50 h" or "Forest Breeze is unscented").\n\nStep 3:If an assumption is wrong, correct it politely.  \n   Then answer the question in a friendly tone.  \n   Mention only the two candles above—we don\'t sell anything else.\n\nUse exactly this output format:\nStep 1:<your reasoning>\nStep 2:<your reasoning>\nStep 3:<your reasoning>\nResponse to user: <final answer>\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Step 1:The user did not mention either of our candles.\nStep 2:The user made no assumptions.\nStep 3:No assumptions were made, so no corrections are needed.\nRespo

In [63]:
chat_history

[HumanMessage(content='Hi! How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Step 1:The user did not mention either of our candles.\nStep 2:The user made no assumptions.\nStep 3:No assumptions were made, so no corrections are needed.\nResponse to user: I'm doing well, thank you for asking! Is there anything I can help you with today? We have Forest Breeze (woodsy scent, 40 h burn, $18) and Vanilla Glow (warm vanilla, 35 h burn, $16).", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I would like to buy 1 Forest Breeze. Can I pay $10?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Step 1:The user mentioned the Forest Breeze candle.\nStep 2:The user assumes the Forest Breeze candle costs $10.\nStep 3:The Forest Breeze candle costs $18, not $10.\n\nResponse to user: I'm happy to hear you like our Forest Breeze candle! Just to clarify, it sells for $18. Would you still like to purchase it?", additional_kwargs={}, respons

In [64]:
user_input = "What was my last question?"

messages = build_messages(chat_history, user_input)

response = model.invoke(messages)

print("\nAssistant:\n", response.content)

chat_history.append(HumanMessage(user_input))
chat_history.append(AIMessage(response.content))


Assistant:
 Step 1:The user did not mention either of our candles.
Step 2:The user made no assumptions about our candles.
Step 3:No assumptions were made, so no corrections are needed.
Response to user: I'm sorry, but I don't have enough memory to recall previous questions. Is there anything I can help you with regarding our Forest Breeze (woodsy scent, 40 h burn, $18) or Vanilla Glow (warm vanilla, 35 h burn, $16) candles?


In [65]:
chat_history

[HumanMessage(content='Hi! How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Step 1:The user did not mention either of our candles.\nStep 2:The user made no assumptions.\nStep 3:No assumptions were made, so no corrections are needed.\nResponse to user: I'm doing well, thank you for asking! Is there anything I can help you with today? We have Forest Breeze (woodsy scent, 40 h burn, $18) and Vanilla Glow (warm vanilla, 35 h burn, $16).", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I would like to buy 1 Forest Breeze. Can I pay $10?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Step 1:The user mentioned the Forest Breeze candle.\nStep 2:The user assumes the Forest Breeze candle costs $10.\nStep 3:The Forest Breeze candle costs $18, not $10.\n\nResponse to user: I'm happy to hear you like our Forest Breeze candle! Just to clarify, it sells for $18. Would you still like to purchase it?", additional_kwargs={}, respons

In [66]:
messages

[SystemMessage(content='\nYou are the assistant for a tiny candle shop. \n\nStep 1:Check whether the user mentions either of our candles:\n   • Forest Breeze (woodsy scent, 40 h burn, $18)  \n   • Vanilla Glow (warm vanilla, 35 h burn, $16)\n\nStep 2:List any assumptions the user makes\n   (e.g. "Vanilla Glow lasts 50 h" or "Forest Breeze is unscented").\n\nStep 3:If an assumption is wrong, correct it politely.  \n   Then answer the question in a friendly tone.  \n   Mention only the two candles above—we don\'t sell anything else.\n\nUse exactly this output format:\nStep 1:<your reasoning>\nStep 2:<your reasoning>\nStep 3:<your reasoning>\nResponse to user: <final answer>\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Step 1:The user did not mention either of our candles.\nStep 2:The user made no assumptions.\nStep 3:No assumptions were made, so no corrections are needed.\nRespo

We manually managed memory in this example, but LangChain offers some easier approaches. Feel free to explore further and stay tuned for upcoming tutorials! :)